In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'

In [4]:
pretrained = './log/pggan/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 250
test_batch = 250
lr = 0.1
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_style2/1000shot' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.2
blog_sig = 0.5
jpg_prob = 0.2

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.01
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'style2/1000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss =  loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
#         auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
#         arc.update(auroc, inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 1000] LR: 0.100000
Train | 16/16 | Loss:1.0880 | MainLoss:0.9324 | SPLoss:1.4076 | CLSLoss:1.4923 | top1:50.1600 | AUROC:0.0000
Test | 32/16 | Loss:0.8361 | MainLoss:0.6915 | SPLoss:1.3033 | CLSLoss:1.4289 | top1:55.0000 | AUROC:0.5710
Test | 129/16 | Loss:0.7838 | MainLoss:0.6392 | SPLoss:1.3033 | CLSLoss:1.4289 | top1:97.9595 | AUROC:1.0000

Epoch: [2 | 1000] LR: 0.130000
Train | 16/16 | Loss:0.8118 | MainLoss:0.6908 | SPLoss:1.0695 | CLSLoss:1.4007 | top1:52.2133 | AUROC:0.0000
Test | 32/16 | Loss:0.7882 | MainLoss:0.6920 | SPLoss:0.8241 | CLSLoss:1.3715 | top1:50.6026 | AUROC:0.5737
Test | 129/16 | Loss:0.6097 | MainLoss:0.5136 | SPLoss:0.8241 | CLSLoss:1.3715 | top1:99.8847 | AUROC:1.0000

Epoch: [3 | 1000] LR: 0.160000
Train | 16/16 | Loss:0.7667 | MainLoss:0.6874 | SPLoss:0.6599 | CLSLoss:1.3382 | top1:54.3467 | AUROC:0.0000
Test | 32/16 | Loss:0.7452 | MainLoss:0.6829 | SPLoss:0.4934 | CLSLoss:1.3026 | top1:56.2179 | AUROC:0.5946
Test | 129/16 | Loss:0.3393 | MainL

Test | 129/16 | Loss:130.7483 | MainLoss:0.7495 | SPLoss:1299.9519 | CLSLoss:0.3581 | top1:54.9221 | AUROC:0.5841

Epoch: [24 | 1000] LR: 0.399858
Train | 16/16 | Loss:74.9576 | MainLoss:0.3852 | SPLoss:745.6807 | CLSLoss:0.4358 | top1:83.7333 | AUROC:0.0000
Test | 32/16 | Loss:34.8661 | MainLoss:0.3695 | SPLoss:344.9193 | CLSLoss:0.4733 | top1:83.7436 | AUROC:0.9474
Test | 129/16 | Loss:35.8477 | MainLoss:1.3511 | SPLoss:344.9197 | CLSLoss:0.4733 | top1:50.2866 | AUROC:0.4839

Epoch: [25 | 1000] LR: 0.399833
Train | 16/16 | Loss:22.4154 | MainLoss:0.3699 | SPLoss:220.4109 | CLSLoss:0.4448 | top1:84.1333 | AUROC:0.0000
Test | 32/16 | Loss:22.0054 | MainLoss:0.3250 | SPLoss:216.7620 | CLSLoss:0.4152 | top1:87.1923 | AUROC:0.9509
Test | 129/16 | Loss:22.5113 | MainLoss:0.8309 | SPLoss:216.7620 | CLSLoss:0.4152 | top1:56.4953 | AUROC:0.6460

Epoch: [26 | 1000] LR: 0.399807
Train | 16/16 | Loss:19.7422 | MainLoss:0.3672 | SPLoss:193.7091 | CLSLoss:0.4166 | top1:83.6533 | AUROC:0.0000
Test 

Test | 32/16 | Loss:18.6560 | MainLoss:0.2602 | SPLoss:183.9115 | CLSLoss:0.4691 | top1:89.4231 | AUROC:0.9627
Test | 129/16 | Loss:19.6209 | MainLoss:1.2251 | SPLoss:183.9118 | CLSLoss:0.4691 | top1:54.2118 | AUROC:0.6114

Epoch: [47 | 1000] LR: 0.398792
Train | 16/16 | Loss:11.5977 | MainLoss:0.3713 | SPLoss:112.2231 | CLSLoss:0.4109 | top1:84.2933 | AUROC:0.0000
Test | 32/16 | Loss:8.8656 | MainLoss:0.2612 | SPLoss:85.9993 | CLSLoss:0.4449 | top1:90.7692 | AUROC:0.9677
Test | 129/16 | Loss:9.7300 | MainLoss:1.1256 | SPLoss:85.9993 | CLSLoss:0.4449 | top1:50.3863 | AUROC:0.4824

Epoch: [48 | 1000] LR: 0.398722
Train | 16/16 | Loss:8.4806 | MainLoss:0.3792 | SPLoss:80.9690 | CLSLoss:0.4498 | top1:84.5067 | AUROC:0.0000
Test | 32/16 | Loss:8.1159 | MainLoss:0.8016 | SPLoss:73.1143 | CLSLoss:0.2849 | top1:50.2821 | AUROC:0.7830
Test | 129/16 | Loss:8.2837 | MainLoss:0.9694 | SPLoss:73.1144 | CLSLoss:0.2849 | top1:49.9782 | AUROC:0.3827

Epoch: [49 | 1000] LR: 0.398650
Train | 16/16 | Lo

Test | 129/16 | Loss:63.5483 | MainLoss:1.4474 | SPLoss:620.9468 | CLSLoss:0.6259 | top1:56.8474 | AUROC:0.6654

Epoch: [69 | 1000] LR: 0.039680
Train | 16/16 | Loss:58.5368 | MainLoss:0.1098 | SPLoss:584.2076 | CLSLoss:0.6301 | top1:96.0000 | AUROC:0.0000
Test | 32/16 | Loss:54.5897 | MainLoss:0.1786 | SPLoss:544.0479 | CLSLoss:0.6316 | top1:93.2821 | AUROC:0.9849
Test | 129/16 | Loss:55.9823 | MainLoss:1.5712 | SPLoss:544.0479 | CLSLoss:0.6316 | top1:55.5078 | AUROC:0.6550

Epoch: [70 | 1000] LR: 0.039669
Train | 16/16 | Loss:51.3220 | MainLoss:0.1276 | SPLoss:511.8808 | CLSLoss:0.6315 | top1:95.3867 | AUROC:0.0000
Test | 32/16 | Loss:47.8520 | MainLoss:0.1736 | SPLoss:476.7202 | CLSLoss:0.6317 | top1:93.2821 | AUROC:0.9834
Test | 129/16 | Loss:49.3282 | MainLoss:1.6499 | SPLoss:476.7201 | CLSLoss:0.6317 | top1:53.0841 | AUROC:0.5899

Epoch: [71 | 1000] LR: 0.039657
Train | 16/16 | Loss:44.9737 | MainLoss:0.1108 | SPLoss:448.5657 | CLSLoss:0.6345 | top1:95.8400 | AUROC:0.0000
Test | 

Test | 32/16 | Loss:3.5272 | MainLoss:0.1793 | SPLoss:33.4135 | CLSLoss:0.6534 | top1:93.0385 | AUROC:0.9834
Test | 129/16 | Loss:4.9057 | MainLoss:1.5578 | SPLoss:33.4135 | CLSLoss:0.6534 | top1:56.9097 | AUROC:0.7070

Epoch: [92 | 1000] LR: 0.039372
Train | 16/16 | Loss:3.2601 | MainLoss:0.1007 | SPLoss:31.5282 | CLSLoss:0.6566 | top1:96.3200 | AUROC:0.0000
Test | 32/16 | Loss:3.1258 | MainLoss:0.1716 | SPLoss:29.4771 | CLSLoss:0.6544 | top1:93.5385 | AUROC:0.9831
Test | 129/16 | Loss:4.6881 | MainLoss:1.7338 | SPLoss:29.4772 | CLSLoss:0.6544 | top1:54.5763 | AUROC:0.6600

Epoch: [93 | 1000] LR: 0.039356
Train | 16/16 | Loss:2.9036 | MainLoss:0.1111 | SPLoss:27.8595 | CLSLoss:0.6539 | top1:95.8400 | AUROC:0.0000
Test | 32/16 | Loss:2.7918 | MainLoss:0.1775 | SPLoss:26.0773 | CLSLoss:0.6542 | top1:93.3205 | AUROC:0.9843
Test | 129/16 | Loss:4.1674 | MainLoss:1.5532 | SPLoss:26.0773 | CLSLoss:0.6542 | top1:56.7508 | AUROC:0.7022

Epoch: [94 | 1000] LR: 0.039340
Train | 16/16 | Loss:2.5

Train | 16/16 | Loss:2.0423 | MainLoss:0.1074 | SPLoss:19.2826 | CLSLoss:0.6651 | top1:96.0800 | AUROC:0.0000
Test | 32/16 | Loss:1.9906 | MainLoss:0.1762 | SPLoss:18.0776 | CLSLoss:0.6672 | top1:93.3590 | AUROC:0.9830
Test | 129/16 | Loss:3.4584 | MainLoss:1.6440 | SPLoss:18.0776 | CLSLoss:0.6672 | top1:57.2212 | AUROC:0.7686

Epoch: [115 | 1000] LR: 0.038962
Train | 16/16 | Loss:1.8084 | MainLoss:0.0904 | SPLoss:17.1130 | CLSLoss:0.6694 | top1:96.8800 | AUROC:0.0000
Test | 32/16 | Loss:1.7988 | MainLoss:0.1865 | SPLoss:16.0567 | CLSLoss:0.6685 | top1:93.1154 | AUROC:0.9812
Test | 129/16 | Loss:3.2289 | MainLoss:1.6165 | SPLoss:16.0566 | CLSLoss:0.6685 | top1:57.7788 | AUROC:0.7822

Epoch: [116 | 1000] LR: 0.038942
Train | 16/16 | Loss:1.6430 | MainLoss:0.1051 | SPLoss:15.3122 | CLSLoss:0.6684 | top1:96.1333 | AUROC:0.0000
Test | 32/16 | Loss:1.6331 | MainLoss:0.1814 | SPLoss:14.4501 | CLSLoss:0.6676 | top1:93.2308 | AUROC:0.9814
Test | 129/16 | Loss:3.0660 | MainLoss:1.6143 | SPLoss:

Train | 16/16 | Loss:0.6441 | MainLoss:0.0849 | SPLoss:5.5252 | CLSLoss:0.6645 | top1:96.9333 | AUROC:0.0000
Test | 32/16 | Loss:0.7128 | MainLoss:0.1796 | SPLoss:5.2652 | CLSLoss:0.6677 | top1:93.4744 | AUROC:0.9831
Test | 129/16 | Loss:2.3459 | MainLoss:1.8127 | SPLoss:5.2652 | CLSLoss:0.6677 | top1:55.3645 | AUROC:0.7337

Epoch: [138 | 1000] LR: 0.038453
Train | 16/16 | Loss:0.6435 | MainLoss:0.1304 | SPLoss:5.0644 | CLSLoss:0.6626 | top1:95.1467 | AUROC:0.0000
Test | 32/16 | Loss:0.6670 | MainLoss:0.1741 | SPLoss:4.8636 | CLSLoss:0.6569 | top1:93.4103 | AUROC:0.9832
Test | 129/16 | Loss:2.1860 | MainLoss:1.6931 | SPLoss:4.8636 | CLSLoss:0.6569 | top1:54.7726 | AUROC:0.6914

Epoch: [139 | 1000] LR: 0.038429
Train | 16/16 | Loss:0.5694 | MainLoss:0.0964 | SPLoss:4.6642 | CLSLoss:0.6604 | top1:96.6933 | AUROC:0.0000
Test | 32/16 | Loss:0.6297 | MainLoss:0.1782 | SPLoss:4.4490 | CLSLoss:0.6620 | top1:93.3974 | AUROC:0.9832
Test | 129/16 | Loss:2.1689 | MainLoss:1.7173 | SPLoss:4.4490 |

Train | 16/16 | Loss:0.3047 | MainLoss:0.0945 | SPLoss:2.0343 | CLSLoss:0.6707 | top1:96.9600 | AUROC:0.0000
Test | 32/16 | Loss:0.3883 | MainLoss:0.1826 | SPLoss:1.9897 | CLSLoss:0.6681 | top1:93.0897 | AUROC:0.9844
Test | 129/16 | Loss:1.8926 | MainLoss:1.6870 | SPLoss:1.9897 | CLSLoss:0.6680 | top1:57.2523 | AUROC:0.7628

Epoch: [161 | 1000] LR: 0.037849
Train | 16/16 | Loss:0.3145 | MainLoss:0.1098 | SPLoss:1.9799 | CLSLoss:0.6678 | top1:95.8133 | AUROC:0.0000
Test | 32/16 | Loss:0.3720 | MainLoss:0.1637 | SPLoss:2.0163 | CLSLoss:0.6612 | top1:93.8462 | AUROC:0.9845
Test | 129/16 | Loss:1.9982 | MainLoss:1.7900 | SPLoss:2.0163 | CLSLoss:0.6612 | top1:54.9564 | AUROC:0.7283

Epoch: [162 | 1000] LR: 0.037820
Train | 16/16 | Loss:0.2901 | MainLoss:0.0856 | SPLoss:1.9782 | CLSLoss:0.6639 | top1:96.9867 | AUROC:0.0000
Test | 32/16 | Loss:0.3821 | MainLoss:0.1825 | SPLoss:1.9290 | CLSLoss:0.6670 | top1:93.4487 | AUROC:0.9857
Test | 129/16 | Loss:1.8892 | MainLoss:1.6896 | SPLoss:1.9290 |

Train | 16/16 | Loss:1.8576 | MainLoss:0.0790 | SPLoss:17.7212 | CLSLoss:0.6457 | top1:97.2000 | AUROC:0.0000
Test | 32/16 | Loss:1.8634 | MainLoss:0.1887 | SPLoss:16.6820 | CLSLoss:0.6497 | top1:93.1795 | AUROC:0.9816
Test | 129/16 | Loss:3.3751 | MainLoss:1.7004 | SPLoss:16.6820 | CLSLoss:0.6497 | top1:58.6386 | AUROC:0.8277

Epoch: [184 | 1000] LR: 0.037151
Train | 16/16 | Loss:1.7046 | MainLoss:0.1131 | SPLoss:15.8503 | CLSLoss:0.6466 | top1:95.3067 | AUROC:0.0000
Test | 32/16 | Loss:1.6813 | MainLoss:0.1791 | SPLoss:14.9574 | CLSLoss:0.6454 | top1:93.2949 | AUROC:0.9816
Test | 129/16 | Loss:3.0606 | MainLoss:1.5584 | SPLoss:14.9574 | CLSLoss:0.6454 | top1:59.0000 | AUROC:0.8348

Epoch: [185 | 1000] LR: 0.037118
Train | 16/16 | Loss:1.5207 | MainLoss:0.0917 | SPLoss:14.2253 | CLSLoss:0.6490 | top1:96.9600 | AUROC:0.0000
Test | 32/16 | Loss:1.5393 | MainLoss:0.1897 | SPLoss:13.4315 | CLSLoss:0.6469 | top1:93.0256 | AUROC:0.9810
Test | 129/16 | Loss:2.8569 | MainLoss:1.5073 | SPLoss:

Train | 16/16 | Loss:0.4561 | MainLoss:0.0946 | SPLoss:3.5490 | CLSLoss:0.6539 | top1:96.7733 | AUROC:0.0000
Test | 32/16 | Loss:0.6619 | MainLoss:0.2245 | SPLoss:4.3086 | CLSLoss:0.6530 | top1:92.0256 | AUROC:0.9761
Test | 129/16 | Loss:1.8753 | MainLoss:1.4379 | SPLoss:4.3086 | CLSLoss:0.6530 | top1:61.3489 | AUROC:0.8716

Epoch: [207 | 1000] LR: 0.036363
Train | 16/16 | Loss:0.5646 | MainLoss:0.1392 | SPLoss:4.1893 | CLSLoss:0.6500 | top1:94.2933 | AUROC:0.0000
Test | 32/16 | Loss:0.5885 | MainLoss:0.1762 | SPLoss:4.0583 | CLSLoss:0.6421 | top1:93.0641 | AUROC:0.9812
Test | 129/16 | Loss:1.8546 | MainLoss:1.4423 | SPLoss:4.0583 | CLSLoss:0.6421 | top1:57.4424 | AUROC:0.8257

Epoch: [208 | 1000] LR: 0.036327
Train | 16/16 | Loss:0.4960 | MainLoss:0.0966 | SPLoss:3.9288 | CLSLoss:0.6478 | top1:96.6933 | AUROC:0.0000
Test | 32/16 | Loss:0.5710 | MainLoss:0.1830 | SPLoss:3.8152 | CLSLoss:0.6470 | top1:93.2308 | AUROC:0.9819
Test | 129/16 | Loss:2.0762 | MainLoss:1.6882 | SPLoss:3.8152 |